In [ ]:
## Notebook to evaluate and store the results from ggcnn for a sample imageset
## Last Trial Run: 7th April 2021

In [1]:
import os 
import glob
import sys 
import torch
import numpy as np
import torch.utils.data 
import imageio
import pandas as pd
import time

In [2]:
ggcnn_dir = "/media/gvk/GVK_Elements/WPI/Spring2021/DR_Grasping/CurrentFocus/ggcnn_repo/"
fadi_dir = "/media/gvk/GVK_Elements/github_repo/Ensemble/Experts/Generative/pretrained_weights"
sys.path.append(ggcnn_dir)
sys.path.append(fadi_dir)
from models.common import post_process_output
from utils.dataset_processing import evaluation, grasp 
from utils.data import get_dataset
from utils.dataset_processing.image import DepthImage

In [29]:

dataset_path = "/media/gvk/GVK_Elements/WPI/Spring2021/DR_Grasping/CurrentFocus/Cornell_Dataset/Full_Cornell/*/"
dataset = get_dataset("cornell")
sys.path.insert(0, dataset_path)
test_data = dataset(dataset_path, include_depth = False, include_rgb = True)


Search Path:  /media/gvk/GVK_Elements/WPI/Spring2021/DR_Grasping/CurrentFocus/Cornell_Dataset/Full_Cornell/*/*/pcd*cpos.txt


In [11]:
## Considering one folder!
# folder1_path = "/media/gvk/GVK_Elements/WPI/Spring2021/DR_Grasping/CurrentFocus/Cornell_Dataset/Full_Cornell/data01/"
# dataset = get_dataset("cornell")
# sys.path.insert(0, folder1_path)
# test_data = dataset(folder1_path, include_depth = False, include_rgb = True)


Search Path:  /media/gvk/GVK_Elements/WPI/Spring2021/DR_Grasping/CurrentFocus/Cornell_Dataset/Full_Cornell/data01/*/pcd*cpos.txt


In [30]:
len(test_data.depth_files)

885

In [32]:
## Model Loading:
fadi_depth_dir = "/media/gvk/GVK_Elements/github_repo/Stacking_Cornell/Experts/Generative/pretrained_weights/color/epoch_37_iou_0.80"
depth_net = torch.load(fadi_depth_dir)
net = depth_net

In [33]:
test_comp = test_data
test_data = torch.utils.data.DataLoader(test_data, batch_size = 1, shuffle = False)

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [35]:
##  Trial for each test image
start_cell = time.time()
results = {'correct': 0, 'failed': 0}
iou_quality = {}
with torch.no_grad():

    ## For each test image
    ## Just testing for one single image
    x, y, didx, rot, zoom =  next(iter(test_data))
    idx = 0
    xc = x.to(device)
    yc = [yi.to(device) for yi in y]
    lossd = net.compute_loss(xc, yc)    ## Ensure that your outputs are from a dataloader!

    q_img, ang_img, width_img = post_process_output( lossd['pred']['pos'], lossd['pred']['cos'], lossd['pred']['sin'], lossd['pred']['width'])
    s = evaluation.calculate_iou_match(q_img, ang_img, test_data.dataset.get_gtbb(didx, rot, zoom), no_grasps = 1, grasp_width = width_img)

    if s:
        results['correct'] += 1
    else:
        results['failed'] += 1

    iou_dict = {}
    ## Now to compare and create a dictionary that stores the quality given image name
    x_comp, _, _, _, _ = test_comp[idx]
    #print(torch.allclose(torch.tensor(x_comp), x))
    if torch.allclose(torch.tensor(x_comp), x):
        img_id = test_comp.depth_files[0].split("/")[-1]

        ## We cannot use an image!
        #quality_dict[str(img_id)] = q_img
        image_id = str(img_id).split('.')[0][:-1]
        iou_quality[image_id] = [int(s), np.max(q_img).astype('float32')]
    else:
        print("Dataloader and Comparison X didn't match")

print("Cell Time: ", time.time() - start_cell)

Cell Time:  0.17312932014465332


In [36]:
results = {'correct': 0, 'failed': 0}
start_cell = time.time()
iou_quality = {}
with torch.no_grad():

    ## For each test image
    ## Just testing for one single image
    #x, y, didx, rot, zoom =  next(iter(test_data))
    for idx, (x, y, didx, rot, zoom) in enumerate(test_data):
        xc = x.to(device)
        yc = [yi.to(device) for yi in y]
        lossd = net.compute_loss(xc, yc)

        q_img, ang_img, width_img = post_process_output( lossd['pred']['pos'], lossd['pred']['cos'], lossd['pred']['sin'], lossd['pred']['width'])
        s = evaluation.calculate_iou_match(q_img, ang_img, test_data.dataset.get_gtbb(didx, rot, zoom), no_grasps = 1, grasp_width = width_img)

        if s:
            results['correct'] += 1
        else:
            results['failed'] += 1

        ## Need to store: q_img and img_id
        ## Now to compare and create a dictionary that stores the quality given image name
        x_comp, _, _, _, _ = test_comp[idx]
        
        ## Now, trying to 
        if torch.allclose(torch.tensor(x_comp), x):
            img_id = test_comp.depth_files[idx].split("/")[-1]
            img_id = str(img_id).split('.')[0][:-1]
            ## We cannot use an image!
            #quality_dict[str(img_id)] = q_img
            iou_quality[str(img_id)] = [int(s), np.max(q_img).astype('float32')]
        else:
            print("Dataloader and Comparison X didn't match")
print("Cell Time: ", time.time() - start_cell) ## ~182 s

Cell Time:  182.27391147613525


In [18]:
#print(iou_quality.keys)

{'pcd0100': [1, 1.0517063], 'pcd0101': [1, 1.045923], 'pcd0102': [1, 0.92928237], 'pcd0103': [1, 0.99394953], 'pcd0104': [1, 0.94518286], 'pcd0105': [1, 0.8229798], 'pcd0106': [0, 0.54132044], 'pcd0107': [1, 0.7088535], 'pcd0108': [1, 0.72504544], 'pcd0109': [1, 0.76442176], 'pcd0110': [1, 0.8763616], 'pcd0111': [1, 0.88344646], 'pcd0112': [1, 0.80746746], 'pcd0113': [1, 0.93202347], 'pcd0114': [1, 0.6510281], 'pcd0115': [1, 0.7091662], 'pcd0116': [0, 0.74798334], 'pcd0117': [0, 0.719038], 'pcd0118': [1, 0.74100757], 'pcd0119': [1, 0.862866], 'pcd0120': [1, 0.9704959], 'pcd0121': [1, 0.90548164], 'pcd0122': [1, 0.7081081], 'pcd0123': [1, 0.645414], 'pcd0124': [0, 0.7311125], 'pcd0125': [1, 0.79679567], 'pcd0126': [0, 0.6268924], 'pcd0127': [0, 0.7406696], 'pcd0128': [1, 0.6956099], 'pcd0129': [1, 0.97353715], 'pcd0130': [1, 0.93971324], 'pcd0131': [1, 0.5738125], 'pcd0132': [0, 0.8657288], 'pcd0133': [1, 0.81644785], 'pcd0134': [1, 0.84345424], 'pcd0135': [0, 0.7722464], 'pcd0136': [1,

In [19]:
## To get the image id
img_id = test_comp.rgb_files[1].split("/")[-1]
depth_id = str(img_id).split('.')[0][:-1]
print(depth_id)

rgb_id = str(img_id).split('.')[0][:-1]
print(rgb_id)

pcd0101
pcd0101


In [20]:
# iou_quality

{'pcd0100': [1, 1.0517063],
 'pcd0101': [1, 1.045923],
 'pcd0102': [1, 0.92928237],
 'pcd0103': [1, 0.99394953],
 'pcd0104': [1, 0.94518286],
 'pcd0105': [1, 0.8229798],
 'pcd0106': [0, 0.54132044],
 'pcd0107': [1, 0.7088535],
 'pcd0108': [1, 0.72504544],
 'pcd0109': [1, 0.76442176],
 'pcd0110': [1, 0.8763616],
 'pcd0111': [1, 0.88344646],
 'pcd0112': [1, 0.80746746],
 'pcd0113': [1, 0.93202347],
 'pcd0114': [1, 0.6510281],
 'pcd0115': [1, 0.7091662],
 'pcd0116': [0, 0.74798334],
 'pcd0117': [0, 0.719038],
 'pcd0118': [1, 0.74100757],
 'pcd0119': [1, 0.862866],
 'pcd0120': [1, 0.9704959],
 'pcd0121': [1, 0.90548164],
 'pcd0122': [1, 0.7081081],
 'pcd0123': [1, 0.645414],
 'pcd0124': [0, 0.7311125],
 'pcd0125': [1, 0.79679567],
 'pcd0126': [0, 0.6268924],
 'pcd0127': [0, 0.7406696],
 'pcd0128': [1, 0.6956099],
 'pcd0129': [1, 0.97353715],
 'pcd0130': [1, 0.93971324],
 'pcd0131': [1, 0.5738125],
 'pcd0132': [0, 0.8657288],
 'pcd0133': [1, 0.81644785],
 'pcd0134': [1, 0.84345424],
 'pcd01

In [38]:
first_entry, last_entry = list(iou_quality.keys())[0], list(iou_quality.keys())[-1]
fname = "_".join((model_name, first_entry, last_entry+".txt"))
fdir = os.path.join(save_dir, fname)
fname

'fadi_color_pcd0100_pcd1034.txt'

In [39]:
## Saving the Collected Results!
import pickle
model_name = "fadi_color"
save_dir = "/media/gvk/GVK_Elements/github_repo/Ensemble/Stacking_Test/Stacking_Data/"
first_entry, last_entry = list(iou_quality.keys())[0], list(iou_quality.keys())[-1]
fname = "_".join((model_name, first_entry, last_entry+".txt"))
fdir = os.path.join(save_dir, fname)
with open(fdir, 'wb') as file_pickle:
    pickle.dump(iou_quality, file_pickle)

In [42]:
len(iou_quality)

885